# Parallel Transport Frames scratch work

This is based on a couple things.

It follows the paper, ["Parallel Transport Approach to Curve Framing" by Andrew J. Hanson & Hui Ma](https://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.42.8103), which I first encountered via the [geom-types/src/ptf.org](https://github.com/thi-ng/geom/blob/master/geom-types/src/ptf.org) examples from Karsten Schmidt's work with [thi.ng/geom](https://github.com/thi-ng/geom). If I refer to any page number or section, I'm talking about this paper.

It was also partly an experiment in learning a bit of [JupyterLab](https://github.com/jupyterlab/jupyterlab), which I only tried out after running into some [Julia](https://julialang.org/) problems on Jupyter. I ordinarily also work in [Maxima](http://maxima.sourceforge.net/), but decided to try out [SymPy](https://www.sympy.org/en/index.html). I was rather impressed with both: JupyterLab addresses many of my complaints with Jupyter, and I found SymPy quite capable and part of a more efficient workflow compared to manually transferring results from Maxima into Python.

## TODO items

- Don't require template to be a single closed path
- Figure out why I am leaving one lone vertex at the origin
- Figure out "face_normals didn't match triangles, ignoring!" (if it matters)
- Find a way to start out the trimesh view with things visible
- Figure out why this knot is not actually a [cinquefoil](https://en.wikipedia.org/wiki/Cinquefoil_knot)...

In [1]:
import sympy
from sympy import symbols, cos, sin, diff, lambdify
from sympy.vector import CoordSys3D
import numpy
import quaternion
import stl.mesh
import trimesh

import parallel_transport
import quat

(p,q) [Torus knot](https://en.wikipedia.org/wiki/Torus_knot) as a parametric formula (torus lies in XY plane):

In [2]:
r,p,q,t = symbols("r,p,q,t")
r = cos(q*t)+2
N = CoordSys3D('N')
knot_vec = (r * cos(p*t))*N.i + (r * sin(p*t))*N.j + (-sin(q*t))*N.k

In [3]:
def vec2python(v):
    return sympy.python(N.origin.locate_new('na', v).express_coordinates(N))

In [4]:
print(vec2python(knot_vec))

q = Symbol('q')
t = Symbol('t')
p = Symbol('p')
e = ((cos(q*t) + 2)*cos(p*t), (cos(q*t) + 2)*sin(p*t), -sin(q*t))


In [5]:
print(vec2python(diff(knot_vec,t)))

p = Symbol('p')
q = Symbol('q')
t = Symbol('t')
e = (-p*(cos(q*t) + 2)*sin(p*t) - q*sin(q*t)*cos(p*t), p*(cos(q*t) + 2)*cos(p*t) - q*sin(p*t)*sin(q*t), -q*cos(q*t))


In [6]:
# can I do this without having to make it a Point?
torus52_expr = knot_vec.subs(p, 5).subs(q, 2)
torus52 = sympy.lambdify([t], N.origin.locate_new('na', torus52_expr).express_coordinates(N))
torus52_tan = sympy.lambdify([t], N.origin.locate_new('na', diff(torus52_expr, t)).express_coordinates(N))
#ts = numpy.linspace(0, 2*numpy.pi, 50000)
ts = numpy.linspace(0, 2*numpy.pi, 2000)
points = numpy.array(torus52(ts)).T
points[:10,:]

array([[ 3.        ,  0.        , -0.        ],
       [ 2.99960977,  0.04714521, -0.00628629],
       [ 2.9984392 ,  0.09427692, -0.01257233],
       [ 2.99648866,  0.14138161, -0.01885787],
       [ 2.99375876,  0.18844581, -0.02514266],
       [ 2.99025034,  0.23545602, -0.03142647],
       [ 2.9859645 ,  0.28239879, -0.03770903],
       [ 2.98090257,  0.32926067, -0.0439901 ],
       [ 2.97506613,  0.37602826, -0.05026943],
       [ 2.968457  ,  0.42268818, -0.05654678]])

In [7]:
# analytical tangents:
tans = numpy.array(torus52_tan(ts)).T
tans = tans / numpy.linalg.norm(tans, axis=1)[:,numpy.newaxis]
tans[:10,:]

array([[-0.        ,  0.9912279 , -0.13216372],
       [-0.01640804,  0.99109232, -0.13216196],
       [-0.0328116 ,  0.99068562, -0.1321567 ],
       [-0.04920622,  0.99000792, -0.13214791],
       [-0.06558742,  0.98905939, -0.13213562],
       [-0.08195074,  0.98784029, -0.13211981],
       [-0.09829171,  0.98635095, -0.13210049],
       [-0.11460587,  0.98459179, -0.13207766],
       [-0.13088879,  0.98256327, -0.1320513 ],
       [-0.14713601,  0.98026595, -0.13202143]])

SymPy is failing at the analytical integration, but also [numerical](https://docs.sympy.org/latest/modules/evalf.html#sums-and-integrals). I tried oscillatory quadrature, but it seems that despite the cos/sin everywhere, they're not in the right form.

In [8]:
# tor = parallel_transport.torsion_integral(torus52_expr, t, 0, 2*sympy.pi)
# Works (returns 1.408) but is horrendously slow
#tor_num = tor.evalf(4)
#tor_num

My own numerical approach seems to work:

In [9]:
parallel_transport.torsion_integral_approx(torus52_expr, t, 0, 2*numpy.pi, 0.001)

-1.4071908841197978

Do I want analytical tangents, or approximations? approximations may, oddly enough, be 'more correct' given that I am already approximating the curve.

In [10]:
points = numpy.array(torus52(ts)).T
tan_approx = parallel_transport.approx_tangents(points)

These are rather close regardless:

In [11]:
numpy.mean(numpy.abs(tans - tan_approx))

9.38346943765395e-06

First, make some sort of template I can duplicate. Use a vertex ordering that can go directly into the STL library.

In [12]:
template = numpy.array([
    [0.0, 0.0, 0.0],
    [0.5, 0.0, 0.0],
    [0.5, 0.0, 0.5],
])

and attempt the algorithm from "Parallel Transport to Curve Framing":

In [13]:
# Precompute cross-products, angles, arc length, and angular difference:
t1 = tans
t2 = numpy.roll(tans, -1, axis=0)
cps = numpy.cross(t1, t2)
cps /= numpy.linalg.norm(cps, axis=1)[:,numpy.newaxis]
thetas = numpy.arccos(numpy.sum(t1*t2, axis=1))
ang = parallel_transport.torsion_integral_approx(torus52_expr, t, 0, 2*numpy.pi, 0.001)
print("Angular change: {}, {} deg".format(ang, ang/numpy.pi*180))
total_length = parallel_transport.approx_arc_length(points)
print("Approximate arc length: {}".format(total_length))
# One can add additional twists here if a multiple of a whole rotation:
# ang += 40 * 2*numpy.pi

Angular change: -1.4071908841197978, -80.62609862934731 deg
Approximate arc length: 64.25469846213409


In [14]:
# Build up the actual mesh, correcting spin as we go:
count = len(points)
data = numpy.zeros(count*2*3*len(template), dtype=stl.mesh.Mesh.dtype)
# Each step on the curve involves len(template) vertices which
# must connect to respective vertices of the next step.  This
# requires two triangles for each vertex of the template, and
# each triangle requires 3 vertices.
vectors = data["vectors"]
length = 0.0
# Use q for a running transformation of our template:
q = numpy.quaternion(1, 0, 0, 0)
for i in range(count):
    t1 = tans[i,:]
    b = cps[i]
    theta = thetas[i]
    # Add another transform (note we post-multiply):
    q = quat.rotation_quaternion(b, theta) * q
    length += numpy.linalg.norm(points[i,:] - points[(i+1) % count,:])
    # Correction by arc length (see section 3.1):
    theta2 = -ang * length / total_length
    # Add a correction for twist.  This doesn't update our
    # running transform - it's only for final geometry:
    q_corrected = quat.rotation_quaternion(t1, theta2) * q
    # Connect every vertex in the frame (the template now rotated &
    # shifted) to the 'last' frame location via 2 triangles
    p_adj = quat.conjugate_by(template.copy(), q_corrected) + points[i,:]
    if i == 0:
        first_p_adj = p_adj
    else:
        for j,_ in enumerate(p_adj):
            idx = len(template)*2*3*(i-1) + j*2
            f1,f2 = parallel_transport.gen_faces(
                p_adj[(j+1) % len(template),:],
                p_adj[j,:],
                last_p_adj[(j+1) % len(template),:],
                last_p_adj[j,:],
            )
            vectors[idx] = f1
            vectors[idx+1] = f2
    last_p_adj = p_adj

# Patch up first & last:
for j,_ in enumerate(last_p_adj):
    idx = len(template)*2*3*(count-1) + j*2
    f1,f2 = parallel_transport.gen_faces(
        last_p_adj[j,:],
        last_p_adj[(j+1) % len(template),:],
        first_p_adj[j,:],
        first_p_adj[(j+1) % len(template),:])
    vectors[idx] = f1
    vectors[idx+1] = f2

mesh = stl.mesh.Mesh(data)
mesh_fname = "test_2k.stl"
mesh.save(mesh_fname)
print("done")

done


In [15]:
m = trimesh.load_mesh(mesh_fname)
m.show()

face_normals didn't match triangles, ignoring!
/home/hodapp/.local/lib/python3.6/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
